# First Analysis of Results from Exact Diagonalization

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import hydra
import h5py
import os

from hydra import compose, initialize
from omegaconf import OmegaConf

In [2]:
initialize(version_base=None, config_path="../../conf")
cfg = compose(config_name="config")
print(OmegaConf.to_yaml(cfg))

method:
  name: exact_diagonalization
system:
  L: 4
  N_f: null
  N_ph: 2
  boundary_conditions: periodic
U_sweep:
  photon_number:
    parameters:
      t: 1.0
      g: 0.5
      omega: 10.0
      U_min: 0.0
      U_max: 20.0
      U_points: 100
  entanglement_entropy:
    parameters:
      t: ${U_sweep.photon_number.parameters.t}
      g: ${U_sweep.photon_number.parameters.g}
      omega: ${U_sweep.photon_number.parameters.omega}
      U_min: ${U_sweep.photon_number.parameters.U_min}
      U_max: ${U_sweep.photon_number.parameters.U_max}
      U_points: ${U_sweep.photon_number.parameters.U_points}
omega_sweep:
  photon_number:
    parameters:
      t: 1.0
      U: 3.0
      g: 0.5
      omega_min: 1.0
      omega_max: 100.0
      omega_points: 13
  entanglement_entropy:
    parameters:
      t: ${omega_sweep.photon_number.parameters.t}
      g: ${omega_sweep.photon_number.parameters.g}
      U: ${omega_sweep.photon_number.parameters.U}
      omega_min: ${omega_sweep.photon_number.pa

In [3]:

# root_data_dir = os.path.expanduser(cfg.root_data_dir)
root_data_dir = cfg.root_data_dir
print(f"Root data directory: {root_data_dir}")


Root data directory: /data/xxz_cavity/exact_diagonalization/


In [4]:
# load hdf5 data and print an overview
filename = "ed_omega_sweep_entanglement_entropy.h5"
data_path = os.path.join(root_data_dir, filename)
with h5py.File(data_path, "r") as f:
    print("Keys: %s" % f.keys())
    for key in f.keys():
        print(f"{key}: {f[key].shape}")

Keys: <KeysViewHDF5 ['param_values', 'results']>
param_values: (13,)
results: (13,)


In [6]:
def print_hdf5_tree(filename):
    def print_attrs(name, obj):
        print(name)
        for key, val in obj.attrs.items():
            print(f"  [ATTR] {key}: {val}")
        if isinstance(obj, h5py.Dataset):
            print(f"  [DATASET] shape={obj.shape}, dtype={obj.dtype}")
    with h5py.File(filename, "r") as f:
        f.visititems(print_attrs)

print_hdf5_tree(data_path)


param_values
  [DATASET] shape=(13,), dtype=float64
results
  [DATASET] shape=(13,), dtype=float64


In [2]:
    
def plot_photon_number_vs_omega(omega_list: np.ndarray, photon_numbers: np.ndarray):
    plt.figure()
    plt.plot(omega_list, photon_numbers, label="Photon Number")
    plt.xlabel(r'Photon Frequency $\Omega$')
    plt.ylabel(r'Average Photon Number $\langle N_{ph} \rangle$')
    plt.title("Photon Number vs Photon Frequency")
    plt.legend()
    plt.grid()
    plt.show()

def plot_photon_number_vs_omega_log_log(omega_list: np.ndarray, photon_numbers: np.ndarray):
    plt.figure()
    plt.loglog(omega_list, photon_numbers, label="Photon Number")
    plt.xlabel(r'Photon Frequency $\Omega$')
    plt.ylabel(r'Average Photon Number $\langle N_{ph} \rangle$')
    plt.title("Photon Number vs Photon Frequency (Log-Log Scale)")
    plt.legend()
    plt.grid()
    plt.show()

def plot_entanglement_entropy_vs_omega_log_log(omega_list: np.ndarray, entropies: np.ndarray):
    plt.figure()
    # add a 1/omega**2 guide line in grey
    guide_line = 1 / omega_list**2
    plt.loglog(omega_list, guide_line, 'k--', label=r'$1/\Omega^2$')
    plt.loglog(omega_list, entropies, label="Entanglement Entropy")
    plt.xlabel(r'Photon Frequency $\Omega$')
    plt.ylabel('Entanglement Entropy')
    plt.title("Entanglement Entropy vs Photon Frequency (Log-Log Scale)")
    plt.legend()
    plt.grid()
    plt.show()

def plot_photon_number_vs_U(U_list: np.ndarray, photon_numbers: np.ndarray):
    plt.figure()
    plt.plot(U_list, photon_numbers, label="Photon Number")
    plt.xlabel('Interaction Strength U')
    plt.ylabel(r'Average Photon Number $\langle N_{ph} \rangle$')
    plt.title("Photon Number vs Interaction Strength U")
    plt.legend()
    plt.grid()
    plt.show()

def plot_entanglement_entropy_vs_U(U_list: np.ndarray, entropies: np.ndarray):
    plt.figure()
    plt.plot(U_list, entropies, label="Entanglement Entropy")
    plt.xlabel('Interaction Strength U')
    plt.ylabel('Entanglement Entropy')
    plt.title("Entanglement Entropy vs Interaction Strength U")
    plt.legend()
    plt.grid()
    plt.show()

def plot_ground_state_amplitudes_vs_U(U_list: np.ndarray, ground_states: np.ndarray):
    # plot the absolute square of the ground state coefficients per sites for various U
    # as a colormap. Start at the top of the plot with the lowest U and go down to highest U
    plt.figure()
    plt.imshow(np.abs(ground_states)**2, aspect='auto', extent=[0, len(ground_states[0]), U_list[-1], U_list[0]]) # type: ignore
    plt.colorbar(label='|Coefficient|^2')
    plt.xlabel('Basis State Index')
    plt.ylabel('Interaction Strength U')
    plt.title('Ground State Coefficients vs Interaction Strength U')
    plt.show()


# def plot_ground_state_vs_U(basis: Basis, U_list: np.ndarray, ground_states: np.ndarray):
#     # assume that we have purely fermionic basis for now
#     # take the state with the highest amplitude in each ground state, obtain its binaray representation
#     # and plot its occupation per site vs U
#     L = basis.L
#     dominant_states = []
#     for gs in ground_states:
#         max_idx = np.argmax(np.abs(gs))
#         dominant_state = basis.states[max_idx]
#         dominant_states.append(dominant_state)

#     plt.figure()
#     for site in range(L):
#         occupations = [(state[0] >> site) & 1 for state in dominant_states]
#         plt.plot(U_list, occupations, label=f'Site {site}')
#     plt.xlabel('Interaction Strength U')
#     plt.ylabel('Occupation')
#     plt.title('Dominant Ground State Occupation vs Interaction Strength U')
#     plt.legend()
#     plt.grid()
#     plt.show()